In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from zipfile import ZipFile
import cv2
import yaml
from tensorflow.keras.models import model_from_yaml
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [27]:
batch_size = 3
 
''' 搭建模型'''
l=tf.keras.layers
model=Sequential()
 
#第一层卷积和池化
model.add(l.Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding='valid',input_shape=(227,227,3),activation='relu'))
model.add(l.BatchNormalization())
model.add(l.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))
 
#第二层卷积和池化
model.add(l.Conv2D(256,(5,5),(1,1),padding='same',activation='relu'))
model.add(l.BatchNormalization())
model.add(l.MaxPooling2D((3,3),(2,2),padding='valid'))
 
#第三层卷积
model.add(l.Conv2D(384,(3,3),(1,1),'same',activation='relu'))
 
#第四层卷积
model.add(l.Conv2D(384,(3,3),(1,1),'same',activation='relu'))
 
#第五层卷积和池化
model.add(l.Conv2D(256,(3,3),(1,1),'same',activation='relu'))
model.add(l.MaxPooling2D((3,3),(2,2),'valid'))
 
#全连接层
model.add(l.Flatten())
model.add(l.Dense(4096,activation='relu'))
model.add(l.Dropout(0.5))
 
model.add(l.Dense(4096,activation='relu'))
model.add(l.Dropout(0.5))
 
model.add(l.Dense(1000,activation='relu'))
model.add(l.Dropout(0.5))
 
#输出层
model.add(l.Dense(2,activation='softmax'))
model.compile(optimizer='adam', loss="binary_crossentropy",metrics=['accuracy'])
 
'''导入图片数据'''

# ZipFile("/home/dzj/caofeng/image/image_train.zip","r").extractall("/home/dzj/caofeng/image/")
# ZipFile("/home/dzj/caofeng/image/image_test1.zip", "r").extractall("/home/dzj/caofeng/image/")

# image_train = "/home/dzj/caofeng/image/image_train"
# image_test = "/home/dzj/caofeng/image/image_test1"

#利用ImageDataGenerator生成一个batch一个batch的数据
datagen=ImageDataGenerator(samplewise_center=True,rescale=1.0/255)   #samplewise_center:使输入数据的每个样本均值为0,rescale:归一化
 
train_generator = datagen.flow_from_directory(
    'D:/image/dataset/image_train',
    classes=['hg','bhg'],
    target_size=(227, 227),
    class_mode='categorical',
    batch_size=batch_size)

validation_generator = datagen.flow_from_directory(
    'D:/image/dataset/image_test1',
    classes=['hg','bhg'],
    target_size=(227, 227),
    class_mode='categorical',
    batch_size=batch_size)

checkpoint_filepath = 'AlexNet_model2.h5'
callback_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_accuracy', mode='max', save_best_only=True)
callback_learningrate = ReduceLROnPlateau(monitor='val_accuracy', mode='max', min_delta=0.03, patience=3, factor=.5, min_lr=0.00001, verbose=1)
callbacks = [callback_checkpoint, callback_learningrate]

'''开始训练'''
model.fit_generator(generator=train_generator,steps_per_epoch=183/3,epochs=30,validation_data=validation_generator,validation_steps=47/3,callbacks=callbacks)

yaml_string = model.to_yaml()  # 保存模型结构到yaml文件
open('./model_architecture2.yaml', 'w').write(yaml_string)
model.save_weights('./AlexNet_model2.h5')  #保存模型参数

Found 183 images belonging to 2 classes.
Found 47 images belonging to 2 classes.
Epoch 1/30
61/61 [==============================] - 22s 364ms/step - loss: 1.1818 - accuracy: 0.6831 - val_loss: 0.6804 - val_accuracy: 0.6809
Epoch 2/30
61/61 [==============================] - 23s 371ms/step - loss: 0.4845 - accuracy: 0.7486 - val_loss: 0.7260 - val_accuracy: 0.6809
Epoch 3/30
61/61 [==============================] - 23s 380ms/step - loss: 0.3898 - accuracy: 0.8361 - val_loss: 0.6830 - val_accuracy: 0.6809
Epoch 4/30
61/61 [==============================] - ETA: 0s - loss: 0.3912 - accuracy: 0.8852
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
61/61 [==============================] - 23s 375ms/step - loss: 0.3912 - accuracy: 0.8852 - val_loss: 0.5566 - val_accuracy: 0.6809
Epoch 5/30
61/61 [==============================] - 23s 379ms/step - loss: 0.2598 - accuracy: 0.9071 - val_loss: 0.9176 - val_accuracy: 0.6809
Epoch 6/30
61/61 [========================

In [9]:
'''导入模型'''
model = model_from_yaml(open('./model_architecture.yaml').read())
model.load_weights('./AlexNet_model.h5')

In [3]:
import urllib
import cv2
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urllib.request.urlopen(url)
    # bytearray将数据转换成（返回）一个新的字节数组
    # asarray 复制数据，将结构化数据转换成ndarray
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    # cv2.imdecode()函数将数据解码成Opencv图像格式
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # return the image
    return image
imgs=[]
image = url_to_image('https://dpic.tiankong.com/t1/nl/QJ6989364695.jpg?x-oss-process=style/794ws')
# plt.imshow(image)
img = cv2.resize(image,(227,227))
imgs.append(img)
a = np.array(imgs)

In [10]:
'''导入模型'''
model = model_from_yaml(open('./model_architecture.yaml').read())
model.load_weights('./AlexNet_model.h5')
 
'''随便输入一张图片测试一下'''
imgs=[]
img=cv2.imread('D:/image/7d5f69b950ab832c639f03baa659de7.png')
img=cv2.resize(img,(227,227))
imgs.append(img)
a=np.array(imgs)

In [8]:
result=model.predict(a)
idx=np.argmax(result)
 
if idx==0:
    print('the image is hg\n')
else:
    print('the image is bhg\n')

# cv2.imshow("image",img)
# cv2.waitKey(0)

the image is bhg



In [48]:
'''随便输入一张图片测试一下'''
imgs=[]
img=cv2.imread('D:/image/16.jpg')
img=cv2.resize(img,(227,227))
imgs.append(img)
a=np.array(imgs)

In [4]:
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urllib.request.urlopen(url)
    # bytearray将数据转换成（返回）一个新的字节数组
    # asarray 复制数据，将结构化数据转换成ndarray
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    # cv2.imdecode()函数将数据解码成Opencv图像格式
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # return the image
    return image
image = url_to_image('https://dzj-prod-1.oss-cn-shanghai.aliyuncs.com/2021/03/08/60459f23e4b0783a977e073f.png')

In [5]:
image = url_to_image('https://dzj-prod-1.oss-cn-shanghai.aliyuncs.com/2021/03/08/60459f23e4b0783a977e073f.png')
cv2.imshow("Image", image)
cv2.waitKey(0)

-1